In [74]:
%pylab notebook
import numpy as np
import scipy as sp
from scipy.linalg import circulant
import itertools
import os

figpath="./figures/"
!mkdir $figpath

Populating the interactive namespace from numpy and matplotlib
mkdir: ./figures/: File exists


In [96]:
N=40 ; V0=1000
x=np.linspace(-1.,1.,N) ; y=np.linspace(-1.,1.,N) ; h=np.mean(x[1:]-x[:-1]) 

z=[t for t in itertools.product(x,y)]
nx=arange(N) ; zidx=[t for t in itertools.product(nx,nx)]

In [97]:
chrg=np.zeros((N,N),float)

V=np.zeros((N,N),float)

for idx in zidx:
    chrg[idx[0],idx[1]]=(-1)**(idx[0]+idx[1])
    
for i,t1 in enumerate(z):
    for j,t2 in enumerate(z):
        if j!=i:
            d=np.sqrt((t1[0]-t2[0])**2. + (t1[1]-t2[1])**2. )
            V[zidx[i][0],zidx[i][1]]=V[zidx[i][0],zidx[i][1]] + chrg[zidx[j][0],zidx[j][1]]*chrg[zidx[i][0],zidx[i][1]]/d
#V=V0*V/abs(np.max(V))-V0

In [98]:
figure()
imshow(chrg)

figure()
imshow(V)
colorbar()

In [99]:
O=np.zeros((len(z),len(z)),float)
Ov=np.zeros((len(z),len(z)),float)

In [100]:
for i,t1 in enumerate(z):        
    Ov[i,i]=V[zidx[i][0],zidx[i][1]]
    for j,t2 in enumerate(z):
        if np.all(t1==t2):
            O[i,j]=-4./h/h
        if t1[0]!=t2[0] and t1[1]!=t2[1]:
            O[i,j]=0
        elif abs(abs(t1[0]-t2[0])/h-1.)<1.e-10:
            O[i,j]=1./h/h
        elif abs(abs(t1[1]-t2[1])/h-1.)<1.e-10:
            O[i,j]=1./h/h

In [106]:
figure()
imshow(-O,origin="True")
colorbar()

In [107]:
En,vec=np.linalg.eigh(-O+Ov)

In [108]:
ioff()
for idx in range(20):
    vis_vec=np.zeros((N,N),float)
    for i,t1 in enumerate(z):        
        vis_vec[zidx[i][0],zidx[i][1]]=vec[i,idx]
    
    figure()
    imshow(vis_vec,origin="lower")
    title("E" + str(idx) + "=" +str(round(En[idx],3)))
    colorbar()
    savefig(figpath + "eigenstate" + str(idx).zfill(2) + ".jpeg")
    
workdir=os.getcwd()
os.chdir(figpath)
cmd="convert -quality 99 -density 150 -delay 120 -loop 0 eigenstate*.jpeg eigen_movie.gif"
os.system(cmd)
os.system("rm *.jpeg")
os.chdir(workdir)